In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import GPy
import pickle
import math
import json

with open('datapoints_dict.pkl', 'rb') as f:
    data = pickle.load(f)
    
formfactors = []
densities = []

for key in data:
    if not isinstance(data[key]['form_factor'], float) and not isinstance(data[key]['total_density'], float):
        formfactors.append(data[key]['form_factor'])
        densities.append(data[key]['total_density'].to_numpy())
        
formfactors = np.array(formfactors)
densities = np.array(densities)

outlier_column_indexes = [387, 261, 10, 16, 277, 24, 32, 34, 419, 37, 38, 551, 43, 430, 303, 311, 583, 78, 82, 89, 221, 353, 364, 118, 252, 175, 176, 229, 392, 567]
densities = np.delete(densities, outlier_column_indexes)
formfactors = np.delete(formfactors, outlier_column_indexes, axis=0)

 /tmp/ipykernel_306600/2013470714.py:20: VisibleDeprecationWarning:Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [2]:
def get_cutoff_inds(td, slope_cutoff=10, amt_cutoff=5, tail_len=15):
    l = len(td)
    li = 0
    ri = -1
    
    count = 0
    for i in range(l):
        xi = td[i]
        xj = td[i+1]
        kk = (xj[1] - xi[1]) / (xj[0] - xi[0])
        
        if kk > slope_cutoff:
            count += 1
        else:
            count = 0
            
        if count >= amt_cutoff:
            li = max(0, i - tail_len)
            break
            
    count = 0
    for i in range(1, l):
        xi = td[-i]
        xj = td[-i-1]
        kk = (xj[1] - xi[1]) / (xi[0] - xj[0])
        
        if kk > slope_cutoff:
            count += 1
        else:
            count = 0
            
        if count >= amt_cutoff:
            ri = min(-1, -i + tail_len)
            break
            
    return li, ri


cut_dens = []

for d in densities:
    li, ri = get_cutoff_inds(d)
    mi = math.floor(np.abs(li - ri) / 2)
    
    if mi > 0:
        cut_dens.append(d[mi:-mi])
    else:
        cut_dens.append(d)
        
densities = np.array(cut_dens)

 /tmp/ipykernel_306600/1259226132.py:50: VisibleDeprecationWarning:Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [4]:
interpolated_densities = []

for i, d in enumerate(densities):
    d_x = d[:, 0].reshape(-1, 1)
    d_y = d[:, 1].reshape(-1, 1)

    kernel = GPy.kern.RBF(input_dim=1, variance=1, lengthscale=1)
    m = GPy.models.GPRegression(d_x, d_y, kernel)
    m.optimize(messages=True)
    m.optimize_restarts(num_restarts = 5)
    N = 200
    xrange = np.linspace(min(d_x), max(d_x), N)
    ypred = m.predict(xrange)[0]
    interpolated_densities.append(np.hstack((xrange, ypred)))
    
    print(f"\n{i+1}/{len(densities)} done\n")

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s09  0003   1.398961e+05   1.852194e+09 
    00s12  0006   5.024271e+04   6.407064e+07 
    00s16  0009   2.244577e+04   1.813057e+06 
    00s19  0011   1.308517e+04   1.975510e+06 
    00s77  0108   2.608408e+02   8.024662e-07 
Runtime:     00s77
Optimization status: Converged

Optimization restart 1/5, f = 260.84081327818666
Optimization restart 2/5, f = 262.56767801368096
Optimization restart 3/5, f = 469.3481062411719
Optimization restart 4/5, f = 260.8408132773354
Optimization restart 5/5, f = 260.84081328267337

1/583 done

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s06  0002   1.321342e+05   2.899226e+09 
    00s18  0009   1.612285e+04   1.655132e+06 
    00s69  0112   2.945449e+02   2.467096e-06 
Runtime:     00s69
Optimization status: Converged

Optimization restart 1/5, f = 294.54493510093687
Optimization restart 2/5, f = 294

 /home/pp/.local/lib/python3.10/site-packages/paramz/optimization/verbose_optimization.py:40: RuntimeWarning:divide by zero encountered in log10



    02s03  0091   2.474320e+02   1.580807e-05 
Runtime:     02s03
Optimization status: Converged

Optimization restart 1/5, f = 223.91715224279292
Optimization restart 2/5, f = 348.62296270911764
Optimization restart 3/5, f = 223.46239531725112
Optimization restart 4/5, f = 223.9171522403398
Optimization restart 5/5, f = 223.4623953587607

478/583 done

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0006   3.680000e+04   3.675894e+07 
    00s97  0099   2.261114e+02   1.955247e-08 
Runtime:     00s97
Optimization status: Converged

Optimization restart 1/5, f = 226.1114028276446
Optimization restart 2/5, f = 226.11140283982107
Optimization restart 3/5, f = 226.11140283938587
Optimization restart 4/5, f = 226.11140284202452
Optimization restart 5/5, f = 226.11140683804186

479/583 done

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s10  0003   1.349212e+05   1.655029e+09 
    00s1

In [12]:
data_dict = {}

for i, (ff, iden) in enumerate(list(zip(formfactors, interpolated_densities))):
    data_dict[i] = {'formfactor': ff, 'density': iden}

with open('density_cut.pkl', 'wb') as f:
    pickle.dump(data_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
json_dict = {}

for i, (ff, iden) in enumerate(list(zip(formfactors, interpolated_densities))):
    xs = iden[:, 0]
    ys = iden[:, 1]
    flattened = list(xs) + list(ys)
    
    json_dict[i] = {'formfactor': list(ff), 'density': flattened}

with open('density_cut.json', 'w') as f:
    json.dump(json_dict, f)